In [1]:
# https://grouplens.org/datasets/movielens/100k/
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=84298e58d8215e62566384715266c5e8318985c0c0c5b9e8819785b6946709cb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkContext


In [3]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [4]:
if __name__ == "__main__":
  sc = SparkContext(appName="PythonCollaborativeFiltering")
  !wget "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
  !unzip ml-100k.zip
  data = sc.textFile("ml-100k/u.data")
  #data = sc.textFile("https://grouplens.org/datasets/movielens/100k/u.data")
  ratings = data.map(lambda l: l.split('\t')).map(lambda l: Rating(int(l[0]),int(l[1]), float(l[2])))
  rank = 10
  numIterations = 10
  model = ALS.train(rank=rank, iterations=numIterations, ratings=ratings)
  testdata = ratings.map(lambda p: (p[0], p[1]))
  predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
  ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
  MSE = ratesAndPreds.map(lambda r: ((r[1][0] - r[1][1])**2)).mean()
  print("Mean Squared Error = " + str(MSE))

  model.save(sc, "target/tmp/myCollaborativeFilter")
  sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")
  sc.stop()

--2024-07-17 02:54:03--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.45MB/s    in 1.4s    

2024-07-17 02:54:06 (3.45 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.ba

In [ ]:
sc.stop()